In [ ]:
#!pip install selenium

In [1]:
#import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
#import json
import pandas as pd

In [2]:
url ='https://www.koreabaseball.com/Schedule/Schedule.aspx?seriesId=0,9'
dpath = r'C:\Users\ahn92\Downloads\chromedriver_win32\chromedriver.exe'

In [3]:
def crawler(url,dpath):
    driver = webdriver.Chrome(executable_path=dpath)
    driver.get(url)
    #xpath1 = '//*[@id="lnb"]/li[1]/a'
    xpathMT = '//*[@id="btnNext"]/img'
    xpathMT2 = '//*[@id="btnPrev"]/img'
    htmlT = []
    for i in range(2,12):
        xpathTM = '//*[@id="contents"]/ul/li['+str(i)+']/a/span'
    #xpathTM = '//*[@id="contents"]/ul/li['+str(2)+']/a/span'
        elementTM = driver.find_element_by_xpath(xpathTM) # xpath link of the elementt
        elementTM.click()
        time.sleep(1)
        html = []
        for j in range(5):
            html.append(driver.page_source)
            elementMT = driver.find_element_by_xpath(xpathMT)
            elementMT.click()
            time.sleep(1)
        htmlT.append(html)
        for j in range(5):
            elementMT2 = driver.find_element_by_xpath(xpathMT2)
            elementMT2.click()
            time.sleep(1)
    time.sleep(1)
    driver.close()
    return(htmlT)

In [4]:
Team_Name = ['SK','두산','한화','넥센','KIA','삼성','롯데','LG','KT','NC']
MM = [3,4,5,6,7]

In [5]:
def mkdf(html):
    soup = BeautifulSoup(html , 'lxml')
    pdate = list(map(lambda x : x.text , soup.findAll('td', attrs={'class':'day'})))
    ptime = list(map(lambda x : x.text , soup.findAll('td', attrs={'class':'time'})))
    match = list(map(lambda x : x.text , soup.findAll('td', attrs={'class':'play'})))
    i = 7
    stadium = []
    while True:
        try:
            stadium.append(list(map(lambda x : x.text , soup.findAll('td')))[i])
            i= i+9
        except:
            break
    df = pd.DataFrame([ptime,pdate,match,stadium]).T
    df.columns = ['time','date','match','stadium']
    return(df)

In [6]:
def mkdflst():
    dflst = []
    for i in list(range(len(Team_Name))):
        dflst2 = []
        for j in list(range(len(MM))):
            dflst2.append(mkdf(htmlT[i][j]))
        dflst.append(dflst2)
    return(dflst)

In [7]:
htmlT = crawler(url,dpath)

In [9]:
dflst = mkdflst()

In [10]:
for i in list(range(len(Team_Name))):
    df = pd.concat([dflst[i][0],dflst[i][1],dflst[i][2],dflst[i][3],dflst[i][4]])
    df.to_csv('tmp/'+Team_Name[i]+'.csv' , encoding='CP949',index=False)